# Terraform Workload Identity Demo

## Purpose

Repo that allows you to demo the new Workload Identity feature from your laptop (Linux and Mac only for now).

---

## Workload Identity Overview

Currently, credentials used to authenticate providers in TFC/E are generally long-lived credentials set as workspace variables. This presents a significant security risk, even with the advances provided by "variable sets".

With workload identity, users are able to authenticate their Vault, AWS, GCP, or AzureRM [soon] providers using OpenID Connect (OIDC) compliant identity tokens to provide just-in-time credentials and eliminate the need to store long lived secrets on TFC/E.

### Benefits of Workload Identity

- **No more secrets**: Removes the need to store long-lived secrets in TFC/E. Temporary credentials are instead retrieved from a cloud platform on the fly. 

- **No more secret rotation logic**: Credentials issued by cloud platform are temporary and short-lived, removing the need to rotate secrets.

- **Granular permissions control**: Allows for using a cloud platform’s authentication and authorization tools to scope permissions based on TFC/E metadata such as a run’s phase, its workspace, or its organization.

### Workload Identity Setup

The following diagram gives a high level view of the integration between TFC/E, customer agent code, and a cloud platform with workload identity.

Enabling this flow requires the following setup on the customer’s end:

- Configuring trust between TFC/E and the cloud platform
- Creating conditional roles in the cloud platform that filter access based on content in the identity token
- Setting required metadata in workspace variables and variable sets as appropriate (such as the role to assume in the cloud platform)
- Using custom agent code to exchange an identity token from TFC/E for short-lived credentials from the cloud platform

Many of these steps are cloud platform specific and are detailed in separate pages specific to the supported cloud platforms.

## Pre-requisites

1. `Docker` and `Docker Compose` is installed on your local machine
2. `Vault` is installed on your local machine
3. You have done a `terraform login` and have a valid credentials file
4. Your Terraform Cloud organization is enabled for Business tier

---

## Demo

Clone Kalen's repo: `terraform-wlid-demo-repo`

In [ ]:
cd /tmp
git clone https://github.com/hashicorp-se/terraform-wlid-demo-repo

In [ ]:
git fetch

In [ ]:
git pull

In [ ]:
git branch -v -a
git switch feature-multi-stage-role
git branch -l

In [ ]:
cd /tmp/terraform-wlid-demo-repo

### config.env

Edit the `.config.env` file with the name of your Terraform Cloud organization

In [ ]:
cat > .config.env <<EOF
export TFC_ORGANIZATION="pphan"
export VAULT_LICENSE="$(cat ~/vault.hclic)"
EOF
source .config.env

### Prep

In [ ]:
export TF_INPUT=false

In [ ]:
PROJECT_DIR=$(git rev-parse --show-toplevel) && echo $PROJECT_DIR

In [ ]:
if [ "$(uname)" == "Darwin" ]; then
  IP_ADDR=$(ifconfig | grep "inet " | grep -v 127.0.0.1 -m 1 | awk 'match($0, /([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)/) {print substr($0,RSTART,RLENGTH)}')
  export VAULT_DEMO_ADDRESS="http://${IP_ADDR}:8200"
elif [ "$(expr substr $(uname -s) 1 5)" == "Linux" ]; then
  IP_ADDR=$(ip route get 8.8.8.8 | grep src | sed 's/.*src \(.* \)/\1/g' | cut -f1 -d ' ')
  export VAULT_DEMO_ADDRESS="http://${IP_ADDR}:8200"
fi
printf "#==> VAULT_DEMO_ADDRESS: $VAULT_DEMO_ADDRESS" 

## Configure Vault

For this tutorial, we will run the steps ourselves to learn how the sausage was made.

## init terraform

In folder `01-terraform-cloud-pre-req`, we provision the following:

- agent pool: `wlid-agents-01`
- an agent token in the pool
- two Terraform Cloud workspaces:
    - `98-workload-identity-demo-multi-role`
    - `99-workload-identity-demo-read`
    - created for the Vault Identity demo
    - configured for `agent execution` using the pool above
    - configured for the CLI driven workflow
- They are provisioned inside of the TFC organization that you set in the `.config.env` file
- The token and the workspace name are exported as environment variables.

Create terraform `tfvars` file.

In [ ]:
pushd 01-terraform-cloud-pre-req >/dev/null
envsubst <"${PROJECT_DIR}/tmpl/01-terraform.auto.tfvars.tmpl" \
  | tee "${PROJECT_DIR}/01-terraform-cloud-pre-req/terraform.auto.tfvars"
popd > /dev/null

In [ ]:
export TF_DATA_DIR=01-terraform-cloud-pre-req

In [ ]:
terraform -chdir=01-terraform-cloud-pre-req init > /dev/null

In [ ]:
time terraform -chdir=01-terraform-cloud-pre-req apply -auto-approve \
  > /tmp/tf_01_apply.out 2>&1 &

Check the progress

In [ ]:
tail /tmp/tf_01_apply.out

In [ ]:
pushd 01-terraform-cloud-pre-req >/dev/null
export TFC_AGENT_TOKEN=$(terraform output -raw agent_token) >/dev/null
# export TFC_WORKSPACE_NAME=$(terraform output -raw workspace_name)
export TFC_WORKSPACE_NAME_SINGLE=$(terraform output -raw workspace_name_single)
export TFC_WORKSPACE_NAME_MULTI=$(terraform output -raw workspace_name_multi)
popd > /dev/null

printf "TFC_AGENT_TOKEN: $TFC_AGENT_TOKEN
TFC_WORKSPACE_NAME: $TFC_WORKSPACE_NAME 
TFC_WORKSPACE_NAME_SINGLE: $TFC_WORKSPACE_NAME_SINGLE
TFC_WORKSPACE_NAME_MULTI: $TFC_WORKSPACE_NAME_MULTI"

## Start Containers - tfc agent and Vault

Build Docker containers
- based on the `$arg` that was supplied (currently only `vault` has a full demo)
- tfc agent
  - created in Docker using the custom image
  - name: tfc-agent-wlid


In [ ]:
VAULT_VERSION=1.10.4-ent #1.7.6_ent 1.10.4-ent

**NOTE**: You need to run this everytime to make sure tfc-agent gets an updated token.

In [ ]:
cat > docker-compose.yml <<EOF
---
version: "3.8"
networks:
  vpcbr:
    driver: bridge
    ipam:
      config:
      - subnet: 10.5.0.0/16

services:
  tfc_agent_wlid_vault:
    build: tfc-agent-images/vault/
    container_name: tfc_agent
    environment:
      VAULT_ADDR: http://127.0.0.1:8200
      TFC_AGENT_TOKEN: ${TFC_AGENT_TOKEN}
      TFC_AGENT_NAME: "tfc-agent-wild"
    command: vault server -config=/vault/config -dev -dev-ha -dev-transactional
    networks:
      vpcbr:
        ipv4_address: 10.5.0.102

  x-vault: &vault
    image: hashicorp/vault-enterprise:${VAULT_VERSION} #1.7.5_ent, suffic can - or _
    environment:
      &vault-env
      VAULT_ADDR: http://127.0.0.1:8200
      #- VAULT_CLUSTER_INTERFACE=eth0
      VAULT_REDIRECT_INTERFACE: eth0
      #- VAULT_API_ADDR=http://vault01:8200
      VAULT_LOG_LEVEL: debug
      VAULT_LICENSE_PATH: /vault/config/vault.hclic
      VAULT_LICENSE: ${VAULT_LICENSE}
      VAULT_DEV_LISTEN_ADDRESS: 0.0.0.0:8200
      VAULT_DEV_ROOT_TOKEN_ID: root
    command: vault server -config=/vault/config -dev -dev-ha -dev-transactional
    cap_add:
      - IPC_LOCK

  vault_s1:
    <<: *vault
    container_name: vault_s1
    hostname: vault_s1
    environment:
      <<: *vault-env
    volumes:
      - ./vault/config/vault_s1:/vault/config
      - ./vault/config/vault.hclic:/vault/config/vault.hclic
      - /vault/data
      - ./vault/logs/vault_s1:/vault/logs
    ports:
      - "8200:8200/tcp"
    networks:
      vpcbr:
        ipv4_address: 10.5.0.101
  vault_s2:
    <<: *vault
    container_name: vault_s2
    hostname: vault_s2
    environment:
      <<: *vault-env
    volumes:
      - ./vault/config/vault_s2:/vault/config
      - ./vault/config/vault.hclic:/vault/config/vault.hclic
      - /vault/data
      - ./vault/logs/vault_s2:/vault/logs
    ports:
      - "8202:8200/tcp"
    networks:
      vpcbr:
        ipv4_address: 10.5.0.102
    cap_add:
      - NET_RAW
      - NET_ADMIN
  vault_s3:
    <<: *vault
    container_name: vault_s3
    hostname: vault_s3
    environment:
      <<: *vault-env
    volumes:
      - ./vault/config/vault_s3:/vault/config
      - ./vault/config/vault.hclic:/vault/config/vault.hclic
      - /vault/data
      - ./vault/logs/vault_s3:/vault/logs
    ports:
      - "8204:8200/tcp"
    networks:
      vpcbr:
        ipv4_address: 10.5.0.103
EOF

### secret sauce in the tfc agent

We add the following hooks to the remote agents, to have them get the appropriate credentials during each run phase (plan and apply)

In [ ]:
cat tfc-agent-images/vault/hooks/terraform-pre-plan

In [ ]:
cat tfc-agent-images/vault/hooks/terraform-pre-apply

### start tfc agent

In [ ]:
export COMPOSE_PROJECT_NAME=hashi
docker-compose up --build --force-recreate -d tfc_agent_wlid_vault

Confirm that tfc agent is `Waiting for next job`.

In [ ]:
docker ps | grep tfc_agent
docker logs tfc_agent 2>&1 | tail

### init vault server

Vault server is started in dev mode and backgrounded.

In [ ]:
export COMPOSE_PROJECT_NAME=hashi
docker-compose up --force-recreate -d vault_s1

### Verify Vault Server

In [ ]:
docker ps | grep vault

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root

In [ ]:
vault status

## prep vault server

In folder `02-terraform-vault-server-prep`, Terraform configures Vault with the `JWT` auth method to support Terraform Cloud operations.

**Multi-Role Demo**:

- A Policy is created
    - associated with a JWT role named `"vault-wlid-owner"`
    - Capabilities: `create`,`read`,`update`, and `delete` objects under `/secret`
- Variables required for Workload Identity are pushed from this run
    - into Terraform Cloud to the workspace `98-workload-identity-demo-multi-role`
    - that will be used for the demo.
    - They are not marked as sensitive so you can view them if you wanted to.
- These variables are:
    - `TFC_WORKLOAD_IDENTITY_AUDIENCE`
    - `TFC_VAULT_PLAN_ROLE`
    - `TFC_VAULT_APPLY_ROLE`
    - `VAULT_ADDR`
- This demo will use the read only role for the `Plan` phase and the owner role for the `Apply` phase

**Read-Only Demo**:

- A Policy is created: `jwt-read`
    - associated with a JWT role named "`vault-wlid-read`"
    - to support reading KV data under `/secret` along with auth policies to login
- `KV` Data is written to the Vault server at the path `/secret/`
- Variables required for Workload Identity are pushed from this run
    - into Terraform Cloud to the `99-workload-identity-demo-single` workspace
    - that will be used for the demo.
- They are not marked as sensitive so you can view them if you wanted to.
- These variables are:
    - `TFC_WORKLOAD_IDENTITY_AUDIENCE`
    - `TFC_VAULT_RUN_ROLE`
    - `VAULT_ADDR`
- This demo will only use 1 Role to login to Vault for both the `Plan` and `Apply` phase



In [ ]:
pushd 02-terraform-vault-server-prep >/dev/null
envsubst <"${PROJECT_DIR}/tmpl/02-terraform.auto.tfvars.tmpl" \
    | tee "${PROJECT_DIR}/02-terraform-vault-server-prep/terraform.auto.tfvars"
popd >/dev/null

In [ ]:
export TF_CONFIG_DIR=02-terraform-vault-server-prep

### Provision

In [ ]:
terraform -chdir=$TF_CONFIG_DIR init > /dev/null

In [ ]:
terraform -chdir=$TF_CONFIG_DIR apply -auto-approve \
  > /tmp/tf_02_apply.out 2>&1 &

In [ ]:
tail -n 50 /tmp/tf_02_apply.out

### Verify Vault Changes

Confirm that `jwt` auth method is enabled.

In [ ]:
vault auth list

Review `jwt` auth method config

In [ ]:
vault read auth/jwt/config

#### roles

See what `jwt` auth roles were created.

In [ ]:
vault list -format=json auth/jwt/role | jq -r .[]

In [ ]:
for i in $(vault list -format=json auth/jwt/role | jq -r .[]); do
printf "\n#==> Role: $i\n"
vault read auth/jwt/role/$i
done
# vault read auth/jwt/role/vault-wlid-read
# vault read auth/jwt/role/vault-wlid-owner

#### policy

In [ ]:
vault policy list

In [ ]:
vault policy read jwt-read

In [ ]:
vault policy read jwt-secret-owner

### audit

In [ ]:
vault audit enable file file_path=/vault/logs/audit.log log_raw=true

## Customizations on Vault CLI

### Create Vault JWT Auth Role

In [ ]:
vault write auth/jwt/role/vault-wlid-demo-plan -<<EOF
{
  "role_type": "jwt",
  "user_claim": "terraform_full_workspace",
  "token_policies": ["jwt-demo-plan"],
  "bound_audiences": ["vault.workload.identity"],
  "bound_claims_type": "glob",
  "bound_claims": {
    "sub": "organization:pphan:workspace:99-workload-identity-demo:run_phase:plan"
  }
}
EOF

- `policies`: List of policies that will be encoded onto generated tokens. This will control the access that the Vault provider has in your vault instance.

- `bound_audiences`: List of aud values to match against, such as vault.workload.identity. This must match the value for the audience specified in the TFC_WORKLOAD_IDENTITY_AUDIENCE

- `bound_claims_type`: How to interpret values in the claims/values map (bound_claims): can be either string (exact match) or glob (wildcard match). Requires Vault 1.4.0 or above.

    - This should be set to glob if matching claims you wish to partially match, such as sub = "organization:my-org:workspace:my-ws:run_phase:*" when the role should match runs regardless of their run phase

- `bound_claims`: A map of claims from the identity token to values to match against.

    - Note: it is strongly recommended that the sub claim at the minimum is checked as an attribute condition to prevent access by bad actors and other valid tokens issued by TFC/E that are intended for other targets

- `user_claim`: The claim to use to uniquely identify the user, such as terraform_full_workspace . This will be used as the name for the Identity entity alias created due to a successful login.

- `role_type`: Type of the role. This must be set to jwt

### Create Vault Policy

In [ ]:
vault write auth/jwt/role/vault-wlid-demo-plan -<<EOF
{
  "role_type": "jwt",
  "user_claim": "terraform_full_workspace",
  "token_policies": ["jwt-demo-plan"],
  "bound_audiences": ["vault.workload.identity"],
  "bound_claims_type": "glob",
  "bound_claims": {
    "sub": "organization:pphan:workspace:99-workload-identity-demo:run_phase:plan"
  }
}
EOF

In [ ]:
vault read auth/jwt/role/vault-wlid-demo-plan

Confirm secret `rick` has been created under `secret` mount mount.

In [ ]:
vault kv list secret/
vault kv get secret/rick

## demo vault jwt multi

The demo itself takes place in `03-terraform-vault-wlid-demo` and you can view the KV secrets in the state file.

In folder `03-terraform-vault-wlid-demo`, Terraform does a CLI driven workflow to read secrets via Vault

1. Terraform Cloud uses the tfc agent that is running in Docker to do the run.
1. Terraform agent uses the `terraform-pre-plan `hook to authenticate with Vault.
    1. The read-only role `vault-wlid-read` is used to login
1. Once the plan succeeds `terraform-pre-apply` hook is used
    1. The owner role `vault-wlid-owner` is used to login
1. The Vault token has the correct permissions since it can write
1. The `KV` data under `/secret` is written and is output as:
    - a sensitive value: `vault_kv`
    - non-sensitive value as a workaround for the demo: `kv_nonsensitive`
2. The agent uses the environment variables that were pushed from the last step to perform the login to vault
    - `TFC_WORKLOAD_IDENTITY_AUDIENCE`
    - `TFC_VAULT_PLAN_ROLE`
    - `TFC_VAULT_APPLY_ROLE`
    - `VAULT_ADDR`
3. The `KV` data under `/secret` is read and is output as a sensitive value.
    - You can view the secret data via the `terraform.tfstate` file in the directory after the run has completed.

---

In [ ]:
pushd 03-terraform-vault-wlid-multi >/dev/null
envsubst <"${PROJECT_DIR}/tmpl/03-demo-multi-settings.tf.tmpl" \
  | tee "${PROJECT_DIR}/03-terraform-vault-wlid-multi/settings.tf"
# cat ${PROJECT_DIR}/03-terraform-vault-wlid-/settings.tf
if [[ -d .terraform ]]; then
  echo "Terraform already initialized, moving on."
else
  echo "Initializing Terraform"
  terraform init > /dev/null
fi
popd >/dev/null

In [ ]:
pushd 03-terraform-vault-wlid-multi >/dev/null
terraform apply -auto-approve | tee /tmp/tf_03_apply.out
popd >/dev/null

View the sensitive data.

In [ ]:
terraform -chdir=03-terraform-vault-wlid-multi output -json vault_kv \
  | jq .data

4. If you would like to show the data from vault as an output you can uncomment lines `10-18` of the `main.tf` file under `03-terraform-vault-wlid-demo`.

### Verification

Check our audit logs. Note that the policy assigned is `jwt-read`.

In [ ]:
docker exec -it vault_s1 sh -c "cat /vault/logs/audit.log" \
  | grep -i "auth/jwt/login" | grep "jwt-" | tail -n 2 | jq -c 

Run only a plan to confirm that we get read level permissions.

In [ ]:
terraform -chdir=03-terraform-vault-wlid-multi plan

Check our audit logs. Note that the policy assigned is `jwt-read`.

In [ ]:
docker exec -it vault_s1 sh -c "cat /vault/logs/audit.log" \
  | grep -i "auth/jwt/login" | grep "jwt-" | tail -n 2 | jq -c 

Sample for login
```json
{
  "time": "2022-08-09T01:58:31.731898754Z",
  "type": "response",
  "auth": {
    "client_token": "hvs.CAESID_zg3vf8UzDOHh1iux5UEnzoIwOYwW1EeWPbfY95AQJGh4KHGh2cy56NWNaaUl3NzhxNVljaFpIN3o4ZjNXYU4",
    "accessor": "xL06fA20cKpZwx43wvW9sMDF",
    "display_name": "jwt-organization:pphan:workspace:98-workload-identity-demo-multi-role",
    "policies": [
      "default",
      "jwt-read"
    ],
    "token_policies": [
      "default",
      "jwt-read"
    ],
    "metadata": {
      "role": "vault-wlid-read"
    },
    "entity_id": "6c6c518f-62b3-1015-a16a-30c9f9684f40",
    "token_type": "service",
    "token_ttl": 2764800
  },
  "request": {
    "id": "1a3adde6-6d14-c0d6-3305-dec67329faaf",
    "operation": "update",
    "mount_type": "jwt",
    "namespace": {
      "id": "root"
    },
    "path": "auth/jwt/login",
    "data": {
      "jwt": "eyJ0eXAiOiJKV1QiLCJraWQiOiJiZmQ1YjEzZjM4YTc0MjZmNzJhMzEzMTE0NGIwYTFlYjlkMDkzMDI3NTU1NzZmMDdhOTVkNGRkZmM0MTk0ZjM2IiwiYWxnIjoiUlMyNTYifQ.eyJ0ZXJyYWZvcm1fcnVuX3BoYXNlIjoicGxhbiIsInRlcnJhZm9ybV93b3Jrc3BhY2VfaWQiOiJ3cy1IUjlVenEyNzh1YW5YM0dZIiwidGVycmFmb3JtX3dvcmtzcGFjZV9uYW1lIjoiOTgtd29ya2xvYWQtaWRlbnRpdHktZGVtby1tdWx0aS1yb2xlIiwidGVycmFmb3JtX29yZ2FuaXphdGlvbl9pZCI6Im9yZy10ZkxESGFwRmdwaGJoekptIiwidGVycmFmb3JtX29yZ2FuaXphdGlvbl9uYW1lIjoicHBoYW4iLCJ0ZXJyYWZvcm1fcnVuX2lkIjoicnVuLW9nc2czcDFMUFN1UWoyeHciLCJ0ZXJyYWZvcm1fZnVsbF93b3Jrc3BhY2UiOiJvcmdhbml6YXRpb246cHBoYW46d29ya3NwYWNlOjk4LXdvcmtsb2FkLWlkZW50aXR5LWRlbW8tbXVsdGktcm9sZSIsImp0aSI6ImQ5ZTg2ZTZhLTdiNGYtNDJjOS1hMzQ0LTg0ZTg5MjMxOWQ3OCIsImlzcyI6Imh0dHBzOi8vYXBwLnRlcnJhZm9ybS5pbyIsImF1ZCI6InZhdWx0Lndvcmtsb2FkLmlkZW50aXR5IiwiaWF0IjoxNjYwMDEwMzA2LCJuYmYiOjE2NjAwMTAzMDYsImV4cCI6MTY2MDAxNzUwNiwic3ViIjoib3JnYW5pemF0aW9uOnBwaGFuOndvcmtzcGFjZTo5OC13b3JrbG9hZC1pZGVudGl0eS1kZW1vLW11bHRpLXJvbGU6cnVuX3BoYXNlOnBsYW4ifQ.BTJvSFBF_haU3zigXU_evWW19Sa-HN7XEhVJgiLFcll7Ry-zp-BD0sIieTk_yOzX8r3Bv6h8wFux4rlZxnyIWjH4JCmTl8iDjJqKPAFauA61IcnKnU9x6ZfJzvhJbv8z2U7hB8mPbUXJQ_jA9rt1KSQv_mWymy2MNVuNC69xZR1gDT8SoRfcT6MhlR55ALn34UFeSsSML9x-1PxeZNBAzP1v38x7bVp6ebAuF_WVlrg-kYe61Nglz3vRijqppzSO1nDWrXZzWkaEBIp5hgHv64R6_rJanNDhhvccR7PZqDdpuAbEXK5b9N3BJzpOS5-Zv0iSgK3nK7bash7fwnsaLw",
      "role": "vault-wlid-read"
    },
    "remote_address": "10.5.0.1",
    "remote_port": 58732
  },
  "response": {
    "auth": {
      "client_token": "hvs.CAESID_zg3vf8UzDOHh1iux5UEnzoIwOYwW1EeWPbfY95AQJGh4KHGh2cy56NWNaaUl3NzhxNVljaFpIN3o4ZjNXYU4",
      "accessor": "xL06fA20cKpZwx43wvW9sMDF",
      "display_name": "jwt-organization:pphan:workspace:98-workload-identity-demo-multi-role",
      "policies": [
        "default",
        "jwt-read"
      ],
      "token_policies": [
        "default",
        "jwt-read"
      ],
      "metadata": {
        "role": "vault-wlid-read"
      },
      "entity_id": "6c6c518f-62b3-1015-a16a-30c9f9684f40",
      "token_type": "service",
      "token_ttl": 2764800
    },
    "mount_type": "jwt"
  }
}
```

# Cleanup

## Process Details

1. Cleanup will run a destroy in each of the folders that leverages Terraform.
    - It will also remove the `.terraform`,`.terraform.lock.hcl`, and `.tfstate` files after the destroy has been completed.
2. Cleanup will also kill the `vault` process that is running locally for the demo after it has removed all of the configuration changes that it has done.
3.  The Docker containers are all stopped after the agents are no longer needed
4.  Docker images are also removed from the machine during the cleanup process.

---

### Destroy vault jwt

In [ ]:
# pushd 04-terraform-vault-wlid-single >/dev/null
# terraform destroy -auto-approve
# rm -rf .terraform terraform.tfstate terraform.tfstate.backup .terraform.lock.hcl
# popd >/dev/null

In [ ]:
pushd 03-terraform-vault-wlid-multi >/dev/null
terraform destroy -auto-approve > /tmp/tf_03_destroy.out 2>&1
cat /tmp/tf_03_destroy.out
rm -rf .terraform terraform.tfstate terraform.tfstate.backup .terraform.lock.hcl
popd >/dev/null

In [ ]:
# pushd 03-terraform-vault-wlid-demo >/dev/null
# terraform destroy -auto-approve > /tmp/tf_03_destroy.out
# rm -rf .terraform terraform.tfstate terraform.tfstate.backup .terraform.lock.hcl
# popd >/dev/null

### Destroy vault config

In [ ]:
pushd 02-terraform-vault-server-prep >/dev/null
terraform destroy -auto-approve > /tmp/tf_02_destroy.out 2>&1
tail -n 50 /tmp/tf_02_destroy.out
rm -rf .terraform terraform.tfstate terraform.tfstate.backup .terraform.lock.hcl
popd >/dev/null

### Destroy terraform cloud workspace and agent

In [ ]:
pushd 01-terraform-cloud-pre-req >/dev/null
terraform destroy -auto-approve > /tmp/tf_01_destroy.out 2>&1
popd >/dev/null
tail -n 50 /tmp/tf_01_destroy.out

### destroy containers

In [ ]:
# docker container stop $(docker container ls -q --filter name=tfc-agent-wlid)
# docker images | grep tfc-agent-wlid | awk '{print $1 ":" $2}' | xargs docker rmi

In [ ]:
docker stop vault_s1
docker rm vault_s1

In [ ]:
export COMPOSE_PROJECT_NAME=hashi
docker-compose down

## Issues

Currently the cleanup will fail because Terraform Cloud reports tha the agent pool still has workspaces using it. The error happens in the GUI as well as via Terraform. Only work around is to wait and re-run the cleanup or to remove the pool manually later

# TODO

- [ ] AWS Portions of the demo (PRs welcome!)
- [x] AWS Docker image builds
- [ ] GCP Portions of the demo (PRs welcome!)
- [x] GCP Docker image builds
- [ ] Look into cleanup issues with the Terraform Cloud API
- [ ] Binary enumeration for Vault and Docker
- [ ] Credential enumeration function for Terraform Cloud

# Resources

Internal
- RFC: https://docs.google.com/document/d/1O4ytxZ08TfZRStPtDl36o12MxdTAHbMV2-dfYgGsVLQ/edit?pli=1#
- Parent: https://hashicorp.atlassian.net/wiki/spaces/TF/pages/2355527738/Workload+Identity+Overview
- Vault: https://hashicorp.atlassian.net/wiki/spaces/TF/pages/2358379522/Workload+Identity+with+the+Vault+Provider

### Vault namespaces

- is there support for Vault namespaces? (using HCP so need to use admin namespace)
    - yes.. but needs to be set in the agent login script..
    - so you could have the script read the namespace from a workspace var
    - this involves adding a --header "X-Vault-Namespace: $TFC_VAULT_NAMESPACE" (or something similar) to the CURL calls in the agent hooks

## Customization Ideas

```
tee $TF_CONFIG_DIR/custom.tf <<"EOF"

resource "vault_policy" "pol" {
  # namespace = var.namespace
  name      = "jwt-demo-plan"
  policy    = <<-EOT
    path "auth/token/create" {
      capabilities = ["update"]
    }
    path "auth/token/lookup-self" {
      capabilities = ["read"]
    }
    path "secret/*" {
      capabilities = ["read"]
    }
EOT
}

resource "vault_jwt_auth_backend_role" "role" {
  # namespace         = var.namespace
  backend           = "jwt" #vault_jwt_auth_backend.jwt.path
  role_name         = "vault-wlid-demo-plan" #var.role_name
  token_policies    = [vault_policy.pol.name]
  bound_audiences   = var.bound_audiences
  bound_claims_type = var.bound_claims_type
  bound_claims      = {
    sub = "organization:pphan:workspace:${var.tfc_workspace}:run_phase:apply"
  }
  user_claim        = var.user_claim
  role_type         = var.role_type
}

variable "bound_audiences" {
  default = ["vault.workload.identity"]
}

variable "bound_claims_type" {
  default     = "glob"
}

# variable "bound_claims" {
#   default = {
#     sub = "organization:pphan:workspace:${var.tfc_workspace}:run_phase:apply"
#   }
# }

variable "user_claim" {
  default = "terraform_full_workspace"
}

variable "role_type" {
  default = "jwt"
}
EOF
```